# Pythonic SysML Elements

This notebook walks through the approach to making SysML elements act Pythonically while in the PyMBE environment in order to support more intuitive codes that inspect and analyze SysML models.

In [ ]:
import json
import pymbe.api as pm

with open('example_data/sysml_3a.json', 'r') as fp:
    element_raw_data = json.load(fp)

test_model = pm.Model.load(element_raw_data)

## Example: Working with Actions in SysML v2 test case 3a

### Fields for model element attributes

The fields for a given element should be referenceable in a Pythonic way, using the dot operator. In addition, the field name should indicate whether the reference is "inside" the Element or avaiable through a traversal reference. In this case, we use "through" for moving from source to target on a relationship and "reverse" to go from target to source.

In [ ]:
all_action_defs = {ele_id:ele for (ele_id, ele)
                   in test_model.all_non_relationships.items()
                   if ele['@type'] == 'ActionDefinition'} 
all_action_defs

In [ ]:
transfer_action = [ele for (ele_id, ele)
                   in all_action_defs.items() if ele['name'] == "Transfer Torque"][0]

With the object assigned to the variable above, we can now explore the element fields:

In [ ]:
transfer_action.name

In [ ]:
transfer_action.ownedElement

In [ ]:
transfer_action.ownedRelationship

In [ ]:
transfer_action.throughFeatureMembership[0]

### Working with Derived Attributes

SysML v2 has a number of attributes that are derived from the Element's relationship to other Elements. For performance reasons, it may be desirable to evaluate these derivations lazily. It may also be desirable to precache these evaluations or build them up in the background. 

## Example: Working with Clean Bicycle KerML example

The same apporach to model load can work with KerML exports also with some refactoring effort.

In [ ]:
with open('example_data/Clean_Bicycle.json', 'r') as fp:
    element_raw_data = json.load(fp)

In [ ]:
len(element_raw_data)

In [ ]:
extracted_elements = [(lambda d: d.update({'@id': ele['identity']['@id']}) or d)(ele['payload'])
                      for ele in element_raw_data]

In [ ]:
bicycle_model = pm.Model.load(extracted_elements)